In [3]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import pandas as pd
from data import get_data
from time import time
import joblib
pd.set_option('display.max_rows', 20, 
              'display.max_columns', 100)

In [4]:
def get_datas(urls, sheets, types):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        s = pd.read_excel(urls, sheet_name=s)
        s = s[[types, 'description']]
        s['description'] = s['description'].str.lower()
        if i == 0:
            data = s.copy()
        else:
            data = pd.concat([data, s], ignore_index=True)
        i += 1
    data.columns = ['target', 'data']
    data['target'] = data['target'].apply(str)
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

    return data

##  Get data

In [8]:
import pandas as pd

def loads_dataset(urls, sheets):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        df = pd.read_excel(urls, sheet_name=s)
        df = df[[ 'description','section','chapter','heading','sub-heading','tariff']]
        df['section'] = df['section'].map('{:02}'.format)
        #df['chapter'] = df['chapter'].map('{:02}'.format)
    #s['heading'] = s['heading'].apply('{:04}'.format)
    #s['sub-heading'] = s['sub-heading'].map('{:06}'.format)
    #s['tariff'] = s['tariff'].map('{:08}'.format)
        df['description'] = df['description'].str.lower()
    
        if i == 0:
            data = df.copy()
        else:
            data = pd.concat([data, df], ignore_index=True)
        i += 1
        print("Load Data.. "+s+" success")
   # data.columns = ['target', 'data']
  #  data['target'] = data['target'].apply(str)
        load_time = time() - t0
        print("Load dataset time:  %0.3fs" % load_time)

    return data   



In [9]:
urls = 'data/test_hs_code.xlsx'
types = 'section'

sheets = ['8_digit','6_digit','4_digit','2_digit', 'imp', 'exp']

df = loads_dataset(urls, sheets)
df.sample(10)

Load Data.. 8_digit success
Load dataset time:  28.192s
Load Data.. 6_digit success
Load dataset time:  56.967s
Load Data.. 4_digit success
Load dataset time:  85.519s
Load Data.. 2_digit success
Load dataset time:  114.413s
Load Data.. imp success
Load dataset time:  143.670s
Load Data.. exp success
Load dataset time:  172.206s


,description,section,chapter,heading,sub-heading,tariff
15578,photographic (other than cinematographic) imag...,18,90,9008,900890,NaN
83385,"joint, expansion 5in rubber, 16 bar (ccr # 110...",15,73,7307,730799,73079990
127069,"mrch-aa, mdt power cartridge housing : mrch-aa...",15,73,7326,732690,73269099
29202,d145a l.t. liquid dispersant in 53ga dr,07,39,3909,390920,39092090
171330,propane rack : aoru900046-0,15,73,7314,731439,73143900
167188,mini container nrsm-189,17,86,8609,860900,86090090
161314,"faceshield frame, foldback msa.",06,30,3006,300650,30065000
42988,"pony dc 8 1/4 od"" 6 5/8 reg pin x 6 5/8 reg bo...",16,84,8431,843143,84314300
157130,"cartridge, ink 955xl high capacity black",16,84,8443,844399,84439990
171831,r49-c-7m cable 777 mcm male pin,15,73,7318,731829,73182900


In [11]:
print(len(df))
df =df.drop_duplicates()
print(len(df))
df

179892
177945


,description,section,chapter,heading,sub-heading,tariff
0,"horses; live, purebred breeding animals - pure...",01,1,101,10121,01012100
1,"horses; live, other than purebred breeding ani...",01,1,101,10129,01012900
2,asses; live - other,01,1,101,10130,01013090
3,mules and hinnies; live- other,01,1,101,10190,01019000
4,"cattle; live, purebred breeding animals - pure...",01,1,102,10221,01022100
...,...,...,...,...,...,...
179887,"wire brush, hand, with wood handle",20,96,9603,960350,96035000
179888,wypall l20 perforated jumbo roll wiper 2 polls,10,48,4818,481810,48181000
179889,"yellow tape 2"" x 10 ytd",07,39,3919,391990,39199090
179890,zinc it crc,06,34,3402,340290,34029099


In [12]:
#save the DataFrame
print("Save DataFrame")

paths =  'data/df_master.pkl'
joblib.dump(df,paths, compress=1)
print('success')

Save DataFrame
success


## Create Model

In [13]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pandas as pd

from data import get_data
from time import time
import joblib
%config InlineBackend.figure_format='retina'
print(f'pandas version:  {pd.__version__}')
pd.Timestamp.now()

pandas version:  1.1.3


Timestamp('2021-02-15 15:25:04.480643')

In [14]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    string = re.sub(r"<", "", string)
    return string.strip().lower()

In [15]:
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

def create_model(df,names):
    X = []
    for i in range(len(df)):
        X.append(clean_str(df.iloc[i][1]))
    
    y = np.array(df["target"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)



    print("pipeline of feature engineering and model")
    model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

    print("paramater selection")


    parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

    print("Training: ")
   
    t0 = time()
    gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
    gs_clf_svm = gs_clf_svm.fit(X, y)
    print(gs_clf_svm.best_score_)
    print(gs_clf_svm.best_params_)

    model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#fit model with training data
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    print(accuracy_score(y_test, pred))

#save the model
    print("Save Model")

    joblib.dump(model, 'data/model/model_'+names +'.pkl', compress=1)
    print("Create Model.. "+names+" success")


    print (classification_report(y_test, pred))

In [16]:
import re
def clean_string(string,chars=None):
    """
    Cleans a ticker for easier use throughout MoneyTree

    Splits by space and only keeps first bit. Also removes
    any characters that are not letters. Returns as lowercase. / uppercase

    >>> clean_ticker('^VIX')
    'vix'
    >>> clean_ticker('SPX Index')
    'spx'
    """
    pattern = re.compile('[\W_+,\s]')
    #res = pattern.sub('', ticker.split(' ')[0])
   
    res = pattern.sub('', string)
    #print(res)
    if chars =='upper':
        return res.upper()
    else:
        return res.lower()
  



    def clean_strings(strings):
        """
        Maps clean_ticker over tickers. (แยก ticker ออกมาทีละ 1 digit)
        """
        return [clean_string(x) for x in strings]

In [17]:
def create_main_model( types):
    data = ""
    t0 = time()
    a = []
    i = 0
    
    df = joblib.load(paths)
    print("download data...") 
       
    df = df[[types, 'description']]   
        
       
    df.columns = ['target', 'data']
    n = df.isnull().sum()
    print("missing values : ", n )
    df.dropna(inplace=True)
    df['target'] = df['target'].apply(int)
    print("Load Data.. "+types+" success")
        
        #save the data train
    print("Save data train")
    create_model(df,types)
    
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

In [18]:
def create_data( sheets, types):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        df = joblib.load('data/train/df_master_train.pkl')
        print("download data...")
        df = df[df['section'] ==s]
       
        df = df[[types, 'description']]
       # df['description'] = df['description'].str.lower()
        
       
        df.columns = ['target', 'data']
        n = df.isnull().sum()
        print("missing values : ", n )
        df.dropna(inplace=True)
        df['target'] = df['target'].apply(int)
        print("Load Data.. "+s+" success")
        
        #save the data train
        print("Save data train")

        joblib.dump(df, 'data/train/df_master_'+s +'.pkl', compress=1)
        print("Save datatable.. "+s+" success")

    
        create_model(df,s)
    
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

In [19]:
types = "section"

create_main_model( types)

download data...
missing values :  target    0
data      1
dtype: int64
Load Data.. section success
Save data train
pipeline of feature engineering and model
paramater selection
Training: 
0.786804041385994
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}
0.8886203989884799
Save Model
Create Model.. section success
              precision    recall  f1-score   support

           1       1.00      0.99      1.00       108
           2       1.00      1.00      1.00        85
           3       1.00      0.96      0.98        25
           4       0.99      0.99      0.99        69
           5       0.79      0.84      0.81       140
           6       0.89      0.92      0.90       561
           7       0.85      0.87      0.86      1880
           8       0.87      0.85      0.86        47
           9       0.88      0.93      0.90        54
          10       0.86      0.94      0.90       110
          11       0.93      0.92      0.93       531
          12       0.92